In [1]:
import os

In [2]:
%pwd

'd:\\Courses\\ML\\MachineLearning\\ML_Projects\\Computer_Vision\\Kidney_Disease_Classification_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Courses\\ML\\MachineLearning\\ML_Projects\\Computer_Vision\\Kidney_Disease_Classification_Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/KhaledAshrafM/Kidney_Disease_Classification_Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="KhaledAshrafM"
os.environ["MLFLOW_TRACKING_PASSWORD"]="0219c93db4bbd07411aee55c268150037bd535b0"

In [6]:
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [12]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts\data_ingestion\kidney_ct_scan_image",
            mlflow_uri="https://dagshub.com/KhaledAshrafM/Kidney_Disease_Classification_Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE 
        )
        return eval_config

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-24 09:38:53,064: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-24 09:38:53,083: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-24 09:38:53,088: INFO: common: created directory at: artifacts]


Found 2207 images belonging to 2 classes.
138/138 [==============================] - 448s 3s/step - loss: 1.2530 - accuracy: 0.8024
[2024-04-24 09:46:24,288: INFO: common: json file saved at: scores.json]


2024/04/24 09:46:28 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-04-24 09:46:35,109: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\khaled\AppData\Local\Temp\tmpnuwsfyxk\model\data\model\assets
[2024-04-24 09:46:41,026: INFO: builder_impl: Assets written to: C:\Users\khaled\AppData\Local\Temp\tmpnuwsfyxk\model\data\model\assets]


c:\Users\khaled\miniconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2024-04-24 10:05:34,799: WARNING: connectionpool: Retrying (JitteredRetry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))': /KhaledAshrafM/Kidney_Disease_Classification_Project.mlflow/api/2.0/mlflow-artifacts/artifacts/22d4db5e293d4e81b54e2957e51ba859/2ad20db6c2f04a3d9f29125c503d41b1/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2024-04-24 10:06:10,986: WARNING: connectionpool: Retrying (JitteredRetry(total=3, connect=5, read=3, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))': /KhaledAshrafM/Kidney_Disease_Classification_Project.mlflow/api/2.0/mlflow-artifacts/artifacts/22d4db5e293d4e81b54e2957e51ba859/2

RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}